# Preparando el proceso de ingesta de datos.

En este notebook vamos a realizar el proceso de ingesta de datos nuevos para la base de datos y poder así poder tener información continua para realizar un modelado de datos.

En este punto vamos a proceder a trabajar con código que hemos ido desarrollando en los anteriores notebooks.

    - Obtención de datos de la página web, pero aplicado a un determinado tiempo, que será periódico.
    - Ingeniería básica para preparar los .csv necesarios  para la carga de datos en la BBDD.
    - Carga de datos nuevos en la BBDD.

Vamos a intentar reducir el codigo aplicado en relación con los notebooks anteriores. Para ello vamos a recurrir a funciones que generaremos a partir de los codigos anteriores y que estarán en el fichero de [**functions.py**](..\Utils\functions.py)

Es por ello que aquí se verá solo el código sencillo de llamamiento a funciones donde generará primero unos ficheros .csv para después cargar los mismos en la base de datos.

## Descargando los datos para su ingesta

Aquí vamos a desarrollar las funciones necesarias para obtener la información de la página web. Así como las modificaciones que consideramos oportunas para adecuar la información a las tablas creadas en nuestra BBDD.

### Primeramente vamos a convertir en funciones lo que tenemos ahora.

In [1]:
import os
os.chdir(os.path.split(os.getcwd())[0])

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
import os
import pickle
import re
from datetime import datetime
import spacy
from nltk.stem.snowball import SnowballStemmer
from Utils import functions as f

In [ ]:
'''Variables'''

url_principal="https://www.amantis.net/"                        
url_secundaria=url_principal+'productos-amantis/'


date=str(datetime.today().strftime('%y%m%d'))
folder_ingest=r'.\Data'

ext=r'.csv'
scrape='_scrape'
file_product=r'\productos'
file_user=r'\usuarios'
file_comment=r'\comentarios'
file_price=r'\precios'
file_tag=r'\tags'
file_ingest_product=file_product+scrape
file_ingest_comment=file_comment+scrape

'''Mapeo de meses'''
dm_mapping={
    'enero':1, 
    'febrero':2, 
    'marzo':3, 
    'abril':4, 
    'mayo':5,
    'junio':6, 
    'julio':7,
    'agosto':8, 
    'septiembre':9, 
    'octubre':10, 
    'noviembre':11, 
    'diciembre':12,
} 

nombre_listas=['amenities','anal','BDSM','femenino','masculino','juguetes','lenceria','muebles']

product_ingest=folder_ingest+file_ingest_product+'_'+date+ext
comment_ingest=folder_ingest+file_ingest_comment+'_'+date+ext
product_engineer=folder_ingest+file_product+'_'+date+ext
tag_engineer=folder_ingest+file_tag+'_'+date+ext
price_engineer=folder_ingest+file_price+'_'+date+ext
comment_engineer=folder_ingest+file_comment+'_'+date+ext
user_engineer=folder_ingest+file_user+'_'+date+ext


In [ ]:
def spider_amantis(url=url_secundaria,product_ingest=product_ingest,comment_ingest=comment_ingest):
    """
    Obtiene las URLs de los productos para luego extraer la información de cada producto.

    Input:
    - url (str): URL base para la extracción de productos.

    Return:
    - noduplicated_product (pd.DataFrame): DataFrame de productos sin duplicados.
    - noduplicated_comments (pd.DataFrame): DataFrame de comentarios sin duplicados.
    """
    
    pages= np.arange(1,25)  
    '''Listas a generar con la información de los productos'''
    lista_URLs = []
    name=[]
    regular_prices=[]
    new_price=[]
    info=[]
    id=[]
    comentarios=[]
    '''Generamos 2 diccionarios con los datos importantes para ingresar en una BBDD'''

    diccionario_datos_productos={"ID":id,"NAME":name,"INFO":info,"LISTA_URL":lista_URLs,"REGULAR_PRICE":regular_prices,"DISCOUNT_PRICE":new_price}

    diccionario_comentarios_productos={"ID":id,"COMENTARIOS":comentarios}

    '''Listas para generar la información de los comentarios'''
    id_comment=[]
    comments=[]
    date=[]
    ratio=[]
    users=[]
    comment=[]

    print("Empezando a recoger datos de las paginas")
    for page in pages:
        
        if page == 1:
            URL = url
            response = requests.get(url)
            soup = bs(response.text, 'lxml')
            productos = soup.find_all(class_='caption')
            for producto in productos[9:]:
                URL_producto = producto.find('a')['href']
                lista_URLs.append(URL_producto)
            
        else:
            URL = url+'page' + str(page)+'/'
            response = requests.get(URL)
            soup = bs(response.text, 'lxml')
            productos = soup.find_all(class_='caption')
            for producto in productos[9:]:
                URL_producto = producto.find('a')['href']
                lista_URLs.append(URL_producto)
    print("Terminando de recoger los datos de los links de las paginas\nEmpezando a generar las listas de los productos")

    for i in range(len(lista_URLs)):
        id.append(i)

        
    '''Extraemos la información de cada producto existente'''

    for URL in lista_URLs:
        url_product=URL
        response_product = requests.get(url_product)
        soup_product = bs(response_product.text, 'lxml')
        user_comments_product=[]
        date_comments_product=[]
        comments_product=[]
        rating=[]

        titulos=soup_product.find_all("h1",class_="h3")
        for titulo in titulos:
            nombre=titulo.get_text(strip=True)
            name.append(nombre)

        all_price = soup_product.find_all("div", class_="productoPrecio pull-right tdd_precio")                        
        for price_container in all_price:                                                                    
            try:
                special_price = price_container.find("span", class_="productSpecialPrice")
                if special_price:
                    item_price = float(special_price.get_text(strip=True).replace(",", ".").split('€')[0])
                    new_price.append(item_price)
                    regular_price = price_container.find("del").get_text(strip=True)
                    item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                    regular_prices.append(item_regular_price)
                else:
                    regular_price = price_container.find("span").get_text(strip=True)
                    item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                    new_price.append(item_regular_price)
                    regular_prices.append(None)
            except:
                new_price.append(None)
                regular_prices.append(None)

        description=soup_product.find("div", class_="description") 
        information=description.get_text().split('\n')[1:]
        documentation = ''.join(information)
        info.append(documentation)


        '''Vamos a obtener los datos de los comentarios de los usuarios'''
        # print("Cargando los datos de los comentarios")

        all_user_comments = soup_product.find_all("span", class_="name-user") 
        for user_comment in all_user_comments:
            user_comments_product.append(user_comment.get_text(strip=True))

        
        all_dates = soup_product.find_all("span", class_="date")  
        for dates in all_dates:
            dates_text=dates.get_text(strip=True)
            # dates=datetime.strftime(dates, '%dd/%mm/%Y')
            date_comments_product.append(dates_text)
            # date_object = datetime.strptime(date_comments_product)

        all_comments = soup_product.find_all("p")
        for formats in all_comments[-len(date_comments_product):]:
            comments_product.append(formats.get_text(strip=True))

        hearts = soup_product.find_all('div', class_= 'box-description')
        for heart in hearts:
            heart_rating = heart.find_all('span', class_= 'fas fa-heart')
            num_hearts = len(heart_rating)
            rating.append(num_hearts)

        datos = list(zip(date_comments_product,rating, user_comments_product,comments_product ))
        comentarios.append(datos)

    for i, regular_price in enumerate(regular_prices):
        if regular_price is None:
            regular_prices[i] = new_price[i]

    print("Realizando la ingenieria de los datos\nEliminando duplicados de productos")
    productos=pd.DataFrame(diccionario_datos_productos)
    noduplicated_product = productos.drop_duplicates(subset='NAME', keep='first')
    removed_id = productos[productos.duplicated(subset='NAME', keep='first')]['ID']

    print("Tratando los comentarios")

    comentarios_productos=pd.DataFrame(diccionario_comentarios_productos)
    comentarios=pd.DataFrame()
    diccionario={"id":id_comment,"comments":comments}

    for id_product,n_comments in enumerate (comentarios_productos['COMENTARIOS']):
        for i in n_comments:
            id_comment.append(id_product)
            comments.append(i)


    for j in range(len(diccionario['comments'])):
        date.append(diccionario['comments'][j][0])
        ratio.append(diccionario['comments'][j][1])
        users.append(diccionario['comments'][j][2])
        comment.append(diccionario['comments'][j][3])


    comentarios['ID']=pd.Series(id_comment)
    comentarios['DATE']=pd.Series(date)
    comentarios['RATIO']=pd.Series(ratio)
    comentarios['USERS']=pd.Series(users)
    comentarios['COMMENT']=pd.Series(comment)
    
    noduplicated_comments = comentarios[~comentarios['ID'].isin(productos[productos['ID'].isin(removed_id)]['ID'])]
    
    h=input("Quieres salvar los datos?").upper()
    if h=="SI":
        noduplicated_product.to_csv(product_ingest,header=True,index=False)           # Tengo que generar el path correcto
        noduplicated_comments.to_csv(comment_ingest,header=True,index=True)           # Tengo que generar el path correcto
    
    return noduplicated_product,noduplicated_comments



In [ ]:
'''Generando nuevos dataframes'''
def product_engineer(df_product,product_engineer=product_engineer):
    """
    Realiza ingeniería de datos para la creación de la tabla de productos.

    Input:
    - df_product (pd.DataFrame): DataFrame de productos.

    Return:
    - df_product (pd.DataFrame): DataFrame de productos con ingeniería aplicada.
    """
    
    print("Generando el fichero de productos")
    
    # df_product=pd.read_csv(folder_ingest+file_ingest_product+'_'+date+ext)
    df_product['NAME'] = df_product['NAME'].str.replace(r'-(?=\w)', '_')
    df_product[['PRODUCT', 'SLOGAN']] = df_product['NAME'].str.split('[,-.]', 1, expand=True)
    df_product['PRODUCT'] = df_product['PRODUCT'].str.strip()
    df_product['SLOGAN'] = df_product['SLOGAN'].str.strip()
    df_product['CHARACTERISTICS'] = df_product['INFO'].str.split('Ver características y medidas|Características', 1).str[1]
    df_product['DESCRIPTION'] = df_product['INFO'].str.split('Ver características y medidas|Características', 1).str[0].str.strip()
    df_product['CHARACTERISTICS'] = df_product['CHARACTERISTICS'].str.replace('\r', ' ')
    df_product['DESCRIPTION'] = df_product['DESCRIPTION'].str.replace('\r', ' ')

    col_1 = df_product.pop('PRODUCT')
    col_2=df_product.pop('SLOGAN')
    col_3=df_product.pop('DESCRIPTION')
    col_4=df_product.pop('CHARACTERISTICS')

    df_product.drop(columns=['NAME'],inplace=True)
    df_product.drop(columns=['INFO'],inplace=True)

    df_product.insert(loc= 1 , column= 'PRODUCT', value= col_1)
    df_product.insert(loc= 2 , column= 'SLOGAN', value= col_2)
    df_product.insert(loc= 3 , column= 'DESCRIPTION', value= col_3)
    df_product.insert(loc= 4 , column= 'CHARACTERISTICS', value= col_4)
    df_product=df_product.iloc[:,:6]
    # df_product.to_csv(folder_ingest+file_product+'_'+date+ext,header=True,index=False)
    h=input("Quieres salvar los datos?").upper()
    if h=="SI":
        df_product.to_csv(product_engineer,header=True,index=False)           # Tengo que generar el path correcto
        
    return df_product

def tag_engineer(df_tag,nombre_listas=nombre_listas,tag_engineer=tag_engineer):
    """
    Realiza ingeniería de datos para la creación de la tabla de tags.

    Input:
    - df_tag (pd.DataFrame): DataFrame de productos.
    - nombre_lista (Lista): Lista de nombres utilizada para cargar listas desde pickles.

    Return:
    - df_tag (pd.DataFrame): DataFrame de tags de los diversos productos.
    """
    from Utils.functions import eliminacion_acentos,cargar_listas_desde_pickles,aplicar_funcion_a_columna
    print("Generando el fichero de tags")
    # df_tag=pd.read_csv(folder_ingest+file_product+'_'+date+ext)
    df_tag['SLOGAN'] = df_tag['SLOGAN'].str.lower()
    df_tag['DESCRIPTION'] = df_tag['DESCRIPTION'].str.lower()
    # df_tag['SLOGAN'] = df_tag['SLOGAN'].apply(eliminacion_acentos)              # Esto da error, debe de ser porque hay NaN en el campo
    df_tag['DESCRIPTION'] = df_tag['DESCRIPTION'].apply(eliminacion_acentos)
    listas = cargar_listas_desde_pickles(nombre_listas)
    for nombre_lista in listas:
        df_tag = aplicar_funcion_a_columna(df_tag, listas,nombre_lista)
    h=input("Quieres salvar los datos?").upper()
    if h=="SI":
        df_tag.to_csv(tag_engineer,header=True,index=False)           # Tengo que generar el path correcto

    return df_tag

def price_engineer (dataframe,price_engineer=price_engineer):
    """
    Realiza ingeniería de datos para la creación de la tabla de precios.

    Input:
    - dataframe (pd.DataFrame): DataFrame de productos.

    Return:
    - df_prices (pd.DataFrame): DataFrame de precios de los productos.
    """

    print("Generando el fichero de precios")
    # dataframe=pd.read_csv(folder_ingest+file_ingest_product+'_'+date+ext)
    col_1 = dataframe.pop('REGULAR_PRICE')
    col_2=dataframe.pop('DISCOUNT_PRICE')
    col_3=dataframe['ID']
    dataframe.insert(loc= 1 , column= 'ID_PRODUCT', value= col_3)
    dataframe.insert(loc= 2 , column= 'REGULAR_PRICE', value= col_1)
    dataframe.insert(loc= 3 , column= 'DISCOUNT_PRICE', value= col_2)
    date_price=datetime.today().strftime('%y/%m/%d')
    df_prices=dataframe.iloc[:,:4]
    df_prices['FECHA']=date_price
    h=input("Quieres salvar los datos?").upper()
    if h=="SI":
        df_prices.to_csv(price_engineer,header=True,index=False)           # Tengo que generar el path correcto
    return df_prices

def comments_engineer (dataframe,dm_mapping=dm_mapping,comment_engineer=comment_engineer,user_engineer=user_engineer):
    """
    Realiza ingeniería de datos para la creación de la tabla de comentarios.

    Input:
    - dataframe (pd.DataFrame): DataFrame de comentarios.
    - dm_mapping (dict): Mapeo de meses para el tratamiento de fechas.

    Return:
    - df_comments (pd.DataFrame): DataFrame de comentarios con ingeniería aplicada.
    - df_users (pd.DataFrame): DataFrame de usuarios que han realizado comentarios.

    """

    '''Tratando a los Usuarios'''
    print("Generando el fichero de usuarios")

    nombre_count = {}
    count = {}

    for i, row in dataframe.iterrows():
        id = row['ID']
        nombre = row['USERS']
        
        if id not in count:
            count[id] = {}
            
        if nombre in count[id]:
            count[id][nombre] += 1
            nueva_nombre = f"{nombre}_{count[id][nombre]}"
            dataframe.loc[i, 'USERS'] = nueva_nombre
        else:
            count[id][nombre] = 1
    lista_users=dataframe['USERS'].unique()
    mivalor = [ x for x in range(len(lista_users))]             
    lista_users=list(lista_users)                                
    lista_users_code = {k: v for k, v in zip(lista_users, mivalor)}   
    dataframe['ID_USERS']= dataframe['USERS'].map(lista_users_code)
    df_users=pd.DataFrame()
    df_users['ID_USERS']=dataframe['ID_USERS']
    df_users['USERS']=dataframe['USERS']
    df_users.drop_duplicates(subset='ID_USERS', keep='first',inplace=True)



    print("Generando el fichero de comentarios")
    '''Tratando a las Fechas'''

    dataframe['DAY']=dataframe['DATE'].str.split(' ').str.get(1).astype('Int64')
    dataframe['MONTH']=dataframe['DATE'].str.split(' ').str.get(2).str.split(',').str.get(0)
    dataframe['YEAR']=dataframe['DATE'].str.split(' ').str.get(-1).astype('Int64')
    dataframe['MONTH']=dataframe['MONTH'].map(dm_mapping)
    dataframe['DATE'] = pd.to_datetime(dataframe.iloc[:,-3:])
    df_comments=dataframe.iloc[:,:-3]

    col = df_comments.pop('ID_USERS')
    df_comments.drop(columns=['USERS'],inplace=True)
    df_comments.insert(loc= 4 , column= 'ID_USERS', value= col)
    h=input("Quieres salvar los datos?").upper()
    if h=="SI":
        df_users.to_csv(user_engineer,header=True,index=False)           # Tengo que generar el path correcto
        df_comments.to_csv(comment_engineer,header=True,index=False)           # Tengo que generar el path correcto

    return df_comments,df_users





Probando las funciones

In [ ]:
noduplicated_product, noduplicated_comments = spider_amantis(url_secundaria)
df_product = product_engineer(noduplicated_product)    
# df_product.head()
# df_tag=tag_engineer(df_product,nombre_listas)
# df_tag.head()


In [ ]:
df_product.head()

In [ ]:
def tag_engineer(df_tag,nombre_listas=nombre_listas,tag_engineer=tag_engineer):
    """
    Realiza ingeniería de datos para la creación de la tabla de tags.

    Input:
    - df_tag (pd.DataFrame): DataFrame de productos.
    - nombre_lista (Lista): Lista de nombres utilizada para cargar listas desde pickles.

    Return:
    - df_tag (pd.DataFrame): DataFrame de tags de los diversos productos.
    """

    print("Generando el fichero de tags")
    # df_tag=pd.read_csv(folder_ingest+file_product+'_'+date+ext)
    df_tag['SLOGAN'] = df_tag['SLOGAN'].str.lower()
    df_tag['DESCRIPTION'] = df_tag['DESCRIPTION'].str.lower()
    # df_tag['SLOGAN'] = df_tag['SLOGAN'].apply(f.eliminacion_acentos)              # Esto da error, debe de ser porque hay NaN en el campo
    df_tag['DESCRIPTION'] = df_tag['DESCRIPTION'].apply(f.eliminacion_acentos)
    listas = f.cargar_listas_desde_pickles(nombre_listas)
    # for nombre_lista in listas:
    #     df_tag = f.aplicar_funcion_a_columna(df_tag, listas,nombre_lista)
    # h=input("Quieres salvar los datos?").upper()
    # if h=="SI":
    #     df_tag.to_csv(tag_engineer,header=True,index=False)           # Tengo que generar el path correcto

    return df_tag,listas

In [ ]:
df_tag,listas=tag_engineer(df_product)
df_tag.head()
# ,nombre_listas=nombre_listas,tag_engineer=tag_engineer)

In [ ]:
# noduplicated_product, noduplicated_comments = spider_amantis(url_secundaria)
noduplicated_product.head()

**Hay que tener en cuenta que las funciones que dejamos finalmente en el fichero .py sufren modificaciones sobre estas.**

## Modificaciones oportunas para adaptar la información a cargar en la BBDDD.

En este sentido tenemos que entender que la BBDD ya tiene determinados productos cargados y, por lo tanto, con su ID establecidas.

Es por ello que hay que cambiar las ID de la descarga a las ID existentes.

¿Cómo lo vamos a realizar? Vamos a extraer la información de la BBDD en un Dataframe y compararlos para cambiar las ID de los campos *[PRODUCT.product]* y *[USERS.users]*.

También hay que tener en cuenta que la descarga hemos obtenido toda la información de *COMMENTS*, por lo que hay que ver cual es la fecha más reciente de un comentario en la BBDD y solo dejar en el dataframe de COMMENTS los comentarios no guardados. 




#### 1. Reduciendo los comentarios.

¿Qué librerías necesitaremos? 

    - Las librerías para llamar a la BBDD.
    - Las librerías para el manejo de Dataframes.
    - Las librerías para el manejo de Fechas.

In [ ]:
import os
os.chdir(os.path.split(os.getcwd())[0])
folder=os.getcwd()

In [15]:
folder

'd:\\Data_science\\Javier\\Repositorios\\Proyecto_tienda_online\\src'

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime,date
import sqlite3
from Utils import functions as f

In [34]:
df_date=pd.read_csv(r'Data\comentarios_240216.csv')
df_date.head()

,ID,DATE,RATIO,COMMENT,ID_USERS
0,0,2024-02-16,5,Limpia bastante bien y lo deja como nuevo! El ...,0
1,0,2024-02-12,5,Producto muy práctico y eficaz. Fácil de utili...,1
2,0,2024-01-24,5,"Super RAPIDO Y EFICAZ. Una limpieza increible,...",2
3,0,2024-01-24,5,Este peoducto nunca falla y es casi obligatori...,3
4,0,2024-01-06,5,"Genial para limpiar los juguetes eróticos , rá...",4


In [35]:
df_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10380 entries, 0 to 10379
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        10380 non-null  int64 
 1   DATE      10380 non-null  object
 2   RATIO     10380 non-null  int64 
 3   COMMENT   10380 non-null  object
 4   ID_USERS  10380 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 405.6+ KB


In [36]:
df_date['DATE']

0        2024-02-16
1        2024-02-12
2        2024-01-24
3        2024-01-24
4        2024-01-06
            ...    
10375    2018-06-11
10376    2023-10-27
10377    2023-06-21
10378    2022-03-25
10379    2023-11-25
Name: DATE, Length: 10380, dtype: object

In [3]:
dm_mapping={
    'enero':1, 
    'febrero':2, 
    'marzo':3, 
    'abril':4, 
    'mayo':5,
    'junio':6, 
    'julio':7,
    'agosto':8, 
    'septiembre':9, 
    'octubre':10, 
    'noviembre':11, 
    'diciembre':12,
} 

In [37]:
def reengineer_comment(BBDD, df_comment):
    '''Reducción del número de registros del dataframe commentarios para su ingesta en la BBDD
    Input:
    - BBDD (str): Base de datos utilizada.
    - df_comment (Dataframe): Dataframe con los comentarios originales, sin filtrar

    Return:
    - df_comment_filtered (Dataframe): Dataframe con los comentarios filtrados y las fechas convertidas en Datetime.
    '''
    print("Iniciando la conversión de fecha y la reducción de datos de comentarios")

    ''' Conectamos con la base de datos, extraemos la fecha más reciente y la cerramos'''
    conn = sqlite3.connect(BBDD)
    cursor = conn.cursor()
    query='''SELECT MAX(DATE) FROM COMMENT'''
    MAX_date=f.sql_query(query,cursor)
    conn.commit()
    cursor.close()
    conn.close()
    fecha_maxima = MAX_date.iloc[0, 0]
    fecha_maxima = date.fromisoformat(fecha_maxima)
    max_date = pd.to_datetime(fecha_maxima, unit='ns')

    '''Cargamos el dataframe y lo preparamos para su filtro'''
    df_comment['DATE']=pd.to_datetime(df_comment['DATE'])
    df_comment_filtered=df_comment[df_comment['DATE']> max_date]
    print("El número de registros a ingresar es:",len(df_comment_filtered))
    return df_comment_filtered

df_date_filtered=reengineer_comment("Resources/online_shop.db",df_date)




Iniciando la conversión de fecha y la reducción de datos de comentarios
El número de registros a ingresar es: 873


#### 2. Reindexando ***PRODUCTOS*** u ***USUARIOS***.

¿Qué librerías necesitaremos? 

    - Las librerías para llamar a la BBDD.
    - Las librerías para el manejo de Dataframes.


In [1]:
import os
import pandas as pd
import sqlite3
os.chdir(os.path.split(os.getcwd())[0])
folder=os.getcwd()
from Utils import functions as f


In [81]:
df_product=pd.read_csv(r'Data\productos_240216.csv')
df_tags=pd.read_csv(r'Data\tags_240216.csv')
df_users=pd.read_csv(r'Data\usuarios_240216.csv')
df_prices=pd.read_csv(r'Data\precios_240216.csv')
df_comment=pd.read_csv(r'Data\comentarios_240216.csv')

In [5]:
# df_product=pd.read_csv(r'Data\productos_240301.csv')
# df_tag=pd.read_csv(r'Data\tags_240301.csv')
df_product.head()

,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL
0,0,LIMPÍN,limpiador de juguetes eróticos,La búsqueda de sensaciones placenteras es una ...,Envase con pulverizador Contiene 200ml.Sin aro...,https://www.amantis.net/limpin-limpiador-jugue...
1,1,Desliz! Lubricante íntimo de agua 100ml,NaN,Tras muchos años de experiencia en lubricantes...,Bote de 100ml de venta exclusiva en amantis.ne...,https://www.amantis.net/desliz-lubricante-inti...
2,2,TOBOGANE HOT RABBIT,el superventas de amantis ¡mejorado!,Vuelve nuestro vibrador de doble estimulación ...,"Medidas: 19 cm (11 cm insertables) y 3,3 cm/ 2...",https://www.amantis.net/tobogane-hot-rabbit-el...
3,3,BALLENATO,tu vibrador a distancia con aleta móvil y sume...,De las profundidades más húmedas llega BALLENA...,Mando a distancia.14cm de punta a cola. 10cm l...,https://www.amantis.net/ballenato-tu-vibrador-...
4,4,Bacanal Gel Anal monodosis,Pack de 10 uds,Bacanal es un lubricante en gel de base de agu...,Contiene 3 ml x 10 unidadesLubricante íntimo a...,https://www.amantis.net/bacanal-monodosis-pack...


In [66]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               198 non-null    int64 
 1   PRODUCT          198 non-null    object
 2   SLOGAN           164 non-null    object
 3   DESCRIPTION      198 non-null    object
 4   CHARACTERISTICS  183 non-null    object
 5   LISTA_URL        198 non-null    object
dtypes: int64(1), object(5)
memory usage: 9.4+ KB


In [82]:
df_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541 entries, 0 to 540
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               541 non-null    int64 
 1   PRODUCT          541 non-null    object
 2   SLOGAN           464 non-null    object
 3   DESCRIPTION      541 non-null    object
 4   CHARACTERISTICS  492 non-null    object
 5   LISTA_URL        541 non-null    object
 6   amenities        541 non-null    bool  
 7   anal             541 non-null    bool  
 8   BDSM             541 non-null    bool  
 9   femenino         541 non-null    bool  
 10  masculino        541 non-null    bool  
 11  juguetes         541 non-null    bool  
 12  lenceria         541 non-null    bool  
 13  muebles          541 non-null    bool  
dtypes: bool(8), int64(1), object(5)
memory usage: 29.7+ KB


In [8]:
conn = sqlite3.connect("Resources/online_shop.db")
cursor = conn.cursor()
query='''SELECT ID,URL FROM PRODUCT'''
df_bbdd=f.sql_query(query,cursor)
conn.commit()
cursor.close()
conn.close()
df_bbdd.head()

,ID,URL
0,0,https://www.amantis.net/masaboom-el-gran-masaj...
1,1,https://www.amantis.net/massaji-potente-masaje...
2,2,https://www.amantis.net/digit-pro-dedal-vibrad...
3,3,https://www.amantis.net/mini-carnival-mini-mas...
4,4,https://www.amantis.net/lingus-vibrador-sexo-o...


In [9]:
df_bbdd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      561 non-null    int64 
 1   URL     561 non-null    object
dtypes: int64(1), object(1)
memory usage: 8.9+ KB


In [10]:
df_productos_coincidentes = pd.merge(df_bbdd, df_product, left_on="URL", right_on="LISTA_URL")
df_productos_coincidentes.head()

,ID_x,URL,ID_y,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL
0,0,https://www.amantis.net/masaboom-el-gran-masaj...,21,MASABOOM,El gran masajeador sexual,"Este es el juguete para todas, todos y todes. ...",Masajeador con forma de micrófonoMaterial: sil...,https://www.amantis.net/masaboom-el-gran-masaj...
1,1,https://www.amantis.net/massaji-potente-masaje...,22,MASSAJI,Potente masajeador japonés sumergible de silicona,¿Quieres una velada perfecta tras un largo día...,Masajeador Japonés Massaji Material: Silicona ...,https://www.amantis.net/massaji-potente-masaje...
2,2,https://www.amantis.net/digit-pro-dedal-vibrad...,112,DIGIT PRO,dedal vibrador con sujeción,Los dedos siempre han sido grandes aliados de ...,Dedal vibradorMaterial: silicona de grado médi...,https://www.amantis.net/digit-pro-dedal-vibrad...
3,3,https://www.amantis.net/mini-carnival-mini-mas...,207,MINI CARNIVAL,mini_masajeador con cuatro cabezales,¿Conoces a MINI+? Puse hoy es tu día de suerte...,Mini masajeador con cuatro cabezalesCabezales ...,https://www.amantis.net/mini-carnival-mini-mas...
4,4,https://www.amantis.net/lingus-vibrador-sexo-o...,412,LINGÜS,Vibrador para Sexo Oral de amantis,Milenios de sabiduría en el arte del cunniling...,Material: silicona + abs10 modos de vibraciónT...,https://www.amantis.net/lingus-vibrador-sexo-o...


In [11]:
len(df_productos_coincidentes)

370

In [12]:
id_producto_BBDD = df_productos_coincidentes['ID_x'].tolist()
id_producto_actual = df_productos_coincidentes['ID_y'].tolist()

In [13]:
map_product = {k: v for k, v in zip(id_producto_actual,id_producto_BBDD)}
map_product

{21: 0,
 22: 1,
 112: 2,
 207: 3,
 412: 4,
 226: 6,
 25: 7,
 363: 8,
 512: 9,
 1: 10,
 2: 11,
 3: 12,
 12: 13,
 14: 15,
 11: 18,
 559: 19,
 6: 20,
 37: 21,
 46: 22,
 16: 24,
 60: 26,
 5: 27,
 10: 28,
 7: 31,
 17: 32,
 65: 33,
 455: 34,
 162: 35,
 45: 36,
 41: 37,
 173: 38,
 36: 39,
 146: 41,
 31: 43,
 47: 44,
 97: 46,
 19: 47,
 121: 49,
 119: 50,
 98: 52,
 23: 53,
 165: 56,
 63: 57,
 62: 58,
 29: 60,
 178: 62,
 39: 63,
 230: 64,
 91: 65,
 77: 66,
 141: 68,
 160: 73,
 83: 74,
 68: 75,
 268: 77,
 335: 78,
 267: 80,
 57: 82,
 181: 85,
 40: 89,
 74: 91,
 234: 92,
 34: 96,
 86: 97,
 43: 98,
 101: 99,
 88: 100,
 136: 102,
 423: 103,
 13: 106,
 76: 107,
 138: 108,
 293: 111,
 291: 113,
 272: 115,
 67: 116,
 533: 118,
 125: 119,
 269: 122,
 108: 124,
 44: 125,
 64: 126,
 361: 128,
 38: 130,
 176: 131,
 72: 134,
 287: 135,
 73: 136,
 153: 138,
 229: 139,
 180: 141,
 85: 142,
 82: 143,
 212: 147,
 149: 148,
 152: 149,
 387: 150,
 295: 151,
 48: 152,
 296: 153,
 395: 154,
 90: 155,
 315: 156,
 84

In [14]:
max_id_product=df_bbdd['ID'].max()
max_id_product

575

In [15]:
df_product_out = pd.merge(df_bbdd, df_product, left_on="URL", right_on="LISTA_URL",how='right',suffixes=('_',''))
df_product_out = df_product_out[df_product_out['URL'].isnull()]
df_product_out.head(10)

,ID_,URL,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL
4,NaN,NaN,4,Bacanal Gel Anal monodosis,Pack de 10 uds,Bacanal es un lubricante en gel de base de agu...,Contiene 3 ml x 10 unidadesLubricante íntimo a...,https://www.amantis.net/bacanal-monodosis-pack...
8,NaN,NaN,8,DUAL LIPPA,masturbador masculino doble,A muchos nos surge la duda a lo hora de decidi...,"Material: TPE (ciberpiel)Textura: muy blando, ...",https://www.amantis.net/dual-lippa-masturbador...
9,NaN,NaN,9,LIBBO,Anillo vibrador para dedo,¿Quieres darle un nuevo toque a tus relaciones...,"Medidas: 4cm x 1,5cm x 2cmMaterial: silicona m...",https://www.amantis.net/libbo-anillo-vibrador-...
15,NaN,NaN,15,PACHA,Succionador y vibrador punto G,"Como casi todo el mundo sabe, el succionador a...",Vibrador con succión sónicaMedidas: 13 cm larg...,https://www.amantis.net/pacha-succionador-y-vi...
18,NaN,NaN,18,GALGA,elegante arnés de amantis,¿Eres una perra que busca sencillez pero al mi...,Composición: 100% Cuero vegano.Talla: Única.Co...,https://www.amantis.net/galga-elegante-arnes-d...
20,NaN,NaN,20,TOCCATA,Vibrador doble con anilla masajeadora,No vas a poder parar de tocarte con este marav...,Medidas largo: 22 cmMedida máxima diámetro: 3 ...,https://www.amantis.net/toccata-vibrador-doble...
25,NaN,NaN,26,YANTA,anillo doble para pene,"En el sexo también cambiamos de marcha, frenam...",Anilla para peneMaterial: Silicona MédicaMedid...,https://www.amantis.net/yanta-anillo-doble-pene/
26,NaN,NaN,27,FLESH GIRL,6 kilos y 40 cm de piel real disfrutable,FLESH GIRL es una bocanada de aire fresco en t...,Longitud total 40 cm; Altura 14 cm; Ancho 23.5...,https://www.amantis.net/flesh-girl-6-kilos-y-4...
27,NaN,NaN,28,LIBBA,pintalabios vibrador con lengua cosquilleante,La felicidad se esconde en las pequeñas cosas ...,Bala vibradora con lengüitaMaterial: Recubrimi...,https://www.amantis.net/libba-pintalabios-vibr...
29,NaN,NaN,30,Jaula de Castidad Masculina LIGERA,NaN,Mantén tus erecciones enjauladas hasta que qui...,· Material: Plástico acrílico transparente· Pe...,https://www.amantis.net/jaula-de-castidad-masc...


In [16]:
df_product_out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171 entries, 4 to 540
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID_              0 non-null      float64
 1   URL              0 non-null      object 
 2   ID               171 non-null    int64  
 3   PRODUCT          171 non-null    object 
 4   SLOGAN           152 non-null    object 
 5   DESCRIPTION      171 non-null    object 
 6   CHARACTERISTICS  160 non-null    object 
 7   LISTA_URL        171 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 12.0+ KB


In [17]:
id_product_out = df_product_out['ID'].tolist()

id_product_out

[4,
 8,
 9,
 15,
 18,
 20,
 26,
 27,
 28,
 30,
 32,
 35,
 42,
 52,
 53,
 54,
 55,
 58,
 59,
 66,
 69,
 70,
 71,
 75,
 78,
 79,
 87,
 89,
 95,
 99,
 100,
 105,
 107,
 110,
 114,
 117,
 120,
 128,
 131,
 134,
 137,
 142,
 143,
 150,
 158,
 166,
 177,
 182,
 183,
 185,
 186,
 189,
 190,
 197,
 198,
 202,
 203,
 204,
 211,
 213,
 216,
 217,
 222,
 235,
 237,
 239,
 252,
 263,
 264,
 266,
 274,
 275,
 276,
 277,
 278,
 281,
 283,
 299,
 302,
 303,
 304,
 306,
 308,
 309,
 313,
 316,
 319,
 322,
 324,
 328,
 330,
 332,
 344,
 355,
 358,
 359,
 368,
 369,
 370,
 371,
 375,
 376,
 386,
 392,
 396,
 397,
 399,
 400,
 408,
 409,
 411,
 414,
 415,
 416,
 420,
 424,
 426,
 429,
 430,
 438,
 444,
 446,
 451,
 461,
 465,
 466,
 469,
 470,
 471,
 473,
 474,
 475,
 478,
 479,
 480,
 486,
 487,
 488,
 489,
 495,
 496,
 498,
 499,
 500,
 501,
 502,
 504,
 510,
 513,
 515,
 518,
 520,
 526,
 527,
 531,
 532,
 537,
 540,
 541,
 542,
 543,
 550,
 553,
 560,
 563,
 564,
 566,
 567,
 569,
 570,
 575]

In [18]:
df_product_out = df_product_out[df_product.columns]

df_product_out['ID'] = max_id_product + 1 + df_product_out.index
df_product_out.head(10)

,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL
4,580,Bacanal Gel Anal monodosis,Pack de 10 uds,Bacanal es un lubricante en gel de base de agu...,Contiene 3 ml x 10 unidadesLubricante íntimo a...,https://www.amantis.net/bacanal-monodosis-pack...
8,584,DUAL LIPPA,masturbador masculino doble,A muchos nos surge la duda a lo hora de decidi...,"Material: TPE (ciberpiel)Textura: muy blando, ...",https://www.amantis.net/dual-lippa-masturbador...
9,585,LIBBO,Anillo vibrador para dedo,¿Quieres darle un nuevo toque a tus relaciones...,"Medidas: 4cm x 1,5cm x 2cmMaterial: silicona m...",https://www.amantis.net/libbo-anillo-vibrador-...
15,591,PACHA,Succionador y vibrador punto G,"Como casi todo el mundo sabe, el succionador a...",Vibrador con succión sónicaMedidas: 13 cm larg...,https://www.amantis.net/pacha-succionador-y-vi...
18,594,GALGA,elegante arnés de amantis,¿Eres una perra que busca sencillez pero al mi...,Composición: 100% Cuero vegano.Talla: Única.Co...,https://www.amantis.net/galga-elegante-arnes-d...
20,596,TOCCATA,Vibrador doble con anilla masajeadora,No vas a poder parar de tocarte con este marav...,Medidas largo: 22 cmMedida máxima diámetro: 3 ...,https://www.amantis.net/toccata-vibrador-doble...
25,601,YANTA,anillo doble para pene,"En el sexo también cambiamos de marcha, frenam...",Anilla para peneMaterial: Silicona MédicaMedid...,https://www.amantis.net/yanta-anillo-doble-pene/
26,602,FLESH GIRL,6 kilos y 40 cm de piel real disfrutable,FLESH GIRL es una bocanada de aire fresco en t...,Longitud total 40 cm; Altura 14 cm; Ancho 23.5...,https://www.amantis.net/flesh-girl-6-kilos-y-4...
27,603,LIBBA,pintalabios vibrador con lengua cosquilleante,La felicidad se esconde en las pequeñas cosas ...,Bala vibradora con lengüitaMaterial: Recubrimi...,https://www.amantis.net/libba-pintalabios-vibr...
29,605,Jaula de Castidad Masculina LIGERA,NaN,Mantén tus erecciones enjauladas hasta que qui...,· Material: Plástico acrílico transparente· Pe...,https://www.amantis.net/jaula-de-castidad-masc...


In [19]:
df_product_out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171 entries, 4 to 540
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               171 non-null    int64 
 1   PRODUCT          171 non-null    object
 2   SLOGAN           152 non-null    object
 3   DESCRIPTION      171 non-null    object
 4   CHARACTERISTICS  160 non-null    object
 5   LISTA_URL        171 non-null    object
dtypes: int64(1), object(5)
memory usage: 9.4+ KB


In [20]:
id_new_product = df_product_out['ID'].tolist()
map_product_out = {k: v for k, v in zip(id_product_out ,id_new_product)}

map_product_out

{4: 580,
 8: 584,
 9: 585,
 15: 591,
 18: 594,
 20: 596,
 26: 601,
 27: 602,
 28: 603,
 30: 605,
 32: 607,
 35: 610,
 42: 617,
 52: 624,
 53: 625,
 54: 626,
 55: 627,
 58: 630,
 59: 631,
 66: 638,
 69: 641,
 70: 642,
 71: 643,
 75: 647,
 78: 650,
 79: 651,
 87: 657,
 89: 659,
 95: 665,
 99: 668,
 100: 669,
 105: 671,
 107: 673,
 110: 676,
 114: 680,
 117: 683,
 120: 686,
 128: 689,
 131: 692,
 134: 695,
 137: 698,
 142: 703,
 143: 704,
 150: 709,
 158: 717,
 166: 725,
 177: 732,
 182: 737,
 183: 738,
 185: 740,
 186: 741,
 189: 744,
 190: 745,
 197: 747,
 198: 748,
 202: 752,
 203: 753,
 204: 754,
 211: 761,
 213: 763,
 216: 766,
 217: 767,
 222: 770,
 235: 783,
 237: 785,
 239: 787,
 252: 793,
 263: 804,
 264: 805,
 266: 807,
 274: 815,
 275: 816,
 276: 817,
 277: 818,
 278: 819,
 281: 822,
 283: 824,
 299: 840,
 302: 843,
 303: 844,
 304: 845,
 306: 847,
 308: 849,
 309: 850,
 313: 854,
 316: 857,
 319: 860,
 322: 863,
 324: 865,
 328: 869,
 330: 871,
 332: 873,
 344: 885,
 355: 896,

Con esto tenemos 2 diccionarios.

Un diccionario con los indices de los productos coincidentes con la BBDD y un diccionario con los productos que no están en la BBDD.

Utilizaremos este último para filtrar y reducir el número de ingresos en las tablas *PRODUCTS* y *TAGS*

Utilizaremos la suma de los dos diccionarios para las tablas de *PRICES* y *COMMENTS*.

In [21]:
map_product.update(map_product_out)
len(map_product)

541

In [22]:
map_product

{21: 0,
 22: 1,
 112: 2,
 207: 3,
 412: 4,
 226: 6,
 25: 7,
 363: 8,
 512: 9,
 1: 10,
 2: 11,
 3: 12,
 12: 13,
 14: 15,
 11: 18,
 559: 19,
 6: 20,
 37: 21,
 46: 22,
 16: 24,
 60: 26,
 5: 27,
 10: 28,
 7: 31,
 17: 32,
 65: 33,
 455: 34,
 162: 35,
 45: 36,
 41: 37,
 173: 38,
 36: 39,
 146: 41,
 31: 43,
 47: 44,
 97: 46,
 19: 47,
 121: 49,
 119: 50,
 98: 52,
 23: 53,
 165: 56,
 63: 57,
 62: 58,
 29: 60,
 178: 62,
 39: 63,
 230: 64,
 91: 65,
 77: 66,
 141: 68,
 160: 73,
 83: 74,
 68: 75,
 268: 77,
 335: 78,
 267: 80,
 57: 82,
 181: 85,
 40: 89,
 74: 91,
 234: 92,
 34: 96,
 86: 97,
 43: 98,
 101: 99,
 88: 100,
 136: 102,
 423: 103,
 13: 106,
 76: 107,
 138: 108,
 293: 111,
 291: 113,
 272: 115,
 67: 116,
 533: 118,
 125: 119,
 269: 122,
 108: 124,
 44: 125,
 64: 126,
 361: 128,
 38: 130,
 176: 131,
 72: 134,
 287: 135,
 73: 136,
 153: 138,
 229: 139,
 180: 141,
 85: 142,
 82: 143,
 212: 147,
 149: 148,
 152: 149,
 387: 150,
 295: 151,
 48: 152,
 296: 153,
 395: 154,
 90: 155,
 315: 156,
 84

La función quedaría así:

In [38]:
def mapeo_productos(BBDD, df_product):
   '''Función para la conversión de los ID de productos

   Input:
      - BBDD (str): nombre de la BBDD.

      - df_product (dataframe): Dataframe donde se encuentran los productos

   Output:
      - map_product (Dict): Diccionario con los ID mapeados coincidentes.
       
      - map_product_out (Dict): Diccionario con los ID mapeados no coincidentes.
   '''
   print("Vamos a reindexar los productos")


   '''Llamando a la base de datos para extraer información'''
   conn = sqlite3.connect(BBDD)
   cursor = conn.cursor()
   query='''SELECT ID,URL FROM PRODUCT'''
   df_bbdd=f.sql_query(query,cursor)
   conn.commit()
   cursor.close()
   conn.close()

   '''Obteniendo los ID coincidentes'''

   df_products_in = pd.merge(df_bbdd, df_product, left_on="URL", right_on="LISTA_URL")
   id_product_BBDD = df_products_in['ID_x'].tolist()
   id_product_new = df_products_in['ID_y'].tolist()
   map_product = {k: v for k, v in zip(id_product_BBDD, id_product_new)}

   '''Obteniendo los ID NO coincidentes'''
   max_id_product=df_bbdd['ID'].max()
   df_product_out = pd.merge(df_bbdd, df_product, left_on="URL", right_on="LISTA_URL",how='right',suffixes=('_',''))
   df_product_out = df_product_out[df_product_out['URL'].isnull()]
   id_product_out = df_product_out['ID'].tolist()
   df_product_out = df_product_out[df_product.columns]
   df_product_out['ID'] = max_id_product + 1 + df_product_out.index
   id_new_product = df_product_out['ID'].tolist()
   map_product_out = {k: v for k, v in zip(id_new_product,id_product_out )}
   map_product.update(map_product_out)

   return map_product, map_product_out


Vamos a realizar la re-indexación a continuación.

In [42]:
df_product.head()

,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL
0,0,TOK®,el Vibrador Sumergible Más Vendido de amantis,Descubre el encanto y la funcionalidad con TOK...,NaN,https://www.amantis.net/tok-el-vibrador-sumerg...
1,1,MUST,Vibrador Interactivo de Larga Distancia,MUST es un vibrador interactivo de última gene...,Destacadas:Control Remoto a Través de Aplicac...,https://www.amantis.net/must-vibrador-interact...
2,2,TOBOGANE,el vibrador doble más vendido,"Por favor, desabróchense los cinturones de seg...","Vibrador con doble motor. Medidas: 19cm, (11c...",https://www.amantis.net/tobogane-el-vibrador-d...
3,3,KUCHI,minisuccionador sónico de amantis,KUCHI ¡Pero qué monada de succionador de clíto...,Succionador sónicoMaterial: silicona médica y...,https://www.amantis.net/kuchi-minisuccionador-...
4,4,PACHA,Succionador y vibrador punto G,"Como casi todo el mundo sabe, el succionador a...",Vibrador con succión sónicaMedidas: 13 cm larg...,https://www.amantis.net/pacha-succionador-y-vi...


In [23]:
df_product_out.replace({'ID':map_product},inplace=True)
df_product_out.head()

,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL
4,580,Bacanal Gel Anal monodosis,Pack de 10 uds,Bacanal es un lubricante en gel de base de agu...,Contiene 3 ml x 10 unidadesLubricante íntimo a...,https://www.amantis.net/bacanal-monodosis-pack...
8,584,DUAL LIPPA,masturbador masculino doble,A muchos nos surge la duda a lo hora de decidi...,"Material: TPE (ciberpiel)Textura: muy blando, ...",https://www.amantis.net/dual-lippa-masturbador...
9,585,LIBBO,Anillo vibrador para dedo,¿Quieres darle un nuevo toque a tus relaciones...,"Medidas: 4cm x 1,5cm x 2cmMaterial: silicona m...",https://www.amantis.net/libbo-anillo-vibrador-...
15,591,PACHA,Succionador y vibrador punto G,"Como casi todo el mundo sabe, el succionador a...",Vibrador con succión sónicaMedidas: 13 cm larg...,https://www.amantis.net/pacha-succionador-y-vi...
18,594,GALGA,elegante arnés de amantis,¿Eres una perra que busca sencillez pero al mi...,Composición: 100% Cuero vegano.Talla: Única.Co...,https://www.amantis.net/galga-elegante-arnes-d...


In [24]:
df_product_out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171 entries, 4 to 540
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               171 non-null    int64 
 1   PRODUCT          171 non-null    object
 2   SLOGAN           152 non-null    object
 3   DESCRIPTION      171 non-null    object
 4   CHARACTERISTICS  160 non-null    object
 5   LISTA_URL        171 non-null    object
dtypes: int64(1), object(5)
memory usage: 9.4+ KB


In [83]:
df_tags.replace({'ID':map_product},inplace=True)
df_tags.head()

,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL,amenities,anal,BDSM,femenino,masculino,juguetes,lenceria,muebles
0,21,LIMPÍN,limpiador de juguetes eróticos,la busqueda de sensaciones placenteras es una ...,Envase con pulverizador Contiene 200ml.Sin aro...,https://www.amantis.net/limpin-limpiador-jugue...,False,False,True,True,False,True,True,False
1,22,Desliz! Lubricante íntimo de agua 100ml,NaN,tras muchos años de experiencia en lubricantes...,Bote de 100ml de venta exclusiva en amantis.ne...,https://www.amantis.net/desliz-lubricante-inti...,True,False,False,True,False,False,True,False
2,112,TOBOGANE HOT RABBIT,el superventas de amantis ¡mejorado!,vuelve nuestro vibrador de doble estimulacion ...,"Medidas: 19 cm (11 cm insertables) y 3,3 cm/ 2...",https://www.amantis.net/tobogane-hot-rabbit-el...,False,False,False,True,False,True,True,False
3,207,BALLENATO,tu vibrador a distancia con aleta móvil y sume...,de las profundidades mas humedas llega ballena...,Mando a distancia.14cm de punta a cola. 10cm l...,https://www.amantis.net/ballenato-tu-vibrador-...,False,False,False,True,False,True,True,False
4,412,Bacanal Gel Anal monodosis,pack de 10 uds,bacanal es un lubricante en gel de base de agu...,Contiene 3 ml x 10 unidadesLubricante íntimo a...,https://www.amantis.net/bacanal-monodosis-pack...,True,True,False,False,False,False,True,False


In [84]:
df_tags

,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL,amenities,anal,BDSM,femenino,masculino,juguetes,lenceria,muebles
0,21,LIMPÍN,limpiador de juguetes eróticos,la busqueda de sensaciones placenteras es una ...,Envase con pulverizador Contiene 200ml.Sin aro...,https://www.amantis.net/limpin-limpiador-jugue...,False,False,True,True,False,True,True,False
1,22,Desliz! Lubricante íntimo de agua 100ml,NaN,tras muchos años de experiencia en lubricantes...,Bote de 100ml de venta exclusiva en amantis.ne...,https://www.amantis.net/desliz-lubricante-inti...,True,False,False,True,False,False,True,False
2,112,TOBOGANE HOT RABBIT,el superventas de amantis ¡mejorado!,vuelve nuestro vibrador de doble estimulacion ...,"Medidas: 19 cm (11 cm insertables) y 3,3 cm/ 2...",https://www.amantis.net/tobogane-hot-rabbit-el...,False,False,False,True,False,True,True,False
3,207,BALLENATO,tu vibrador a distancia con aleta móvil y sume...,de las profundidades mas humedas llega ballena...,Mando a distancia.14cm de punta a cola. 10cm l...,https://www.amantis.net/ballenato-tu-vibrador-...,False,False,False,True,False,True,True,False
4,412,Bacanal Gel Anal monodosis,pack de 10 uds,bacanal es un lubricante en gel de base de agu...,Contiene 3 ml x 10 unidadesLubricante íntimo a...,https://www.amantis.net/bacanal-monodosis-pack...,True,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,215,MYKONOS,conjunto lencero dulcemente sexy,"muchas veces cuanto mas sexies nos sentimos, m...","Talla: S, M, LMaterial: Nylon y spandexColor: ...",https://www.amantis.net/mykonos-conjunto-lence...,False,False,False,False,False,False,True,False
537,572,SUSU,conjunto lencero con volantes,el sexo es una de las cosas mas divertidas que...,"Talla: S, M, LMaterial: Nylon y spandexColor:...",https://www.amantis.net/susu-conjunto-lencero-...,False,False,False,False,False,False,True,False
538,449,COSQUI,cosquilleador erógeno,la finalidad del encuentro sexual es el placer...,NaN,https://www.amantis.net/cosqui-cosquilleador-e...,False,False,False,False,False,False,True,False
539,574,DITALE Vibrador para dedo,NaN,¿como podemos intensificar sensaciones? las co...,Vibrador de bolsillo con sujeciónMateriales: S...,https://www.amantis.net/ditale-vibrador-dedo/,False,False,False,False,False,True,True,False


In [27]:
nombre_listas=['amenities','anal','BDSM','femenino','masculino','juguetes','lenceria','muebles']
nombre_listas.append('ID')
nombre_listas

['amenities',
 'anal',
 'BDSM',
 'femenino',
 'masculino',
 'juguetes',
 'lenceria',
 'muebles',
 'ID']

In [86]:
df_tag_out = pd.merge(df_product_out, df_tags,left_on="ID", right_on="ID",how='right',suffixes=('_',''))
# df_tag_out=df_tag_out[nombre_listas]
df_tag_out

,ID,PRODUCT_,SLOGAN_,DESCRIPTION_,CHARACTERISTICS_,LISTA_URL_,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL,amenities,anal,BDSM,femenino,masculino,juguetes,lenceria,muebles
0,21,NaN,NaN,NaN,NaN,NaN,LIMPÍN,limpiador de juguetes eróticos,la busqueda de sensaciones placenteras es una ...,Envase con pulverizador Contiene 200ml.Sin aro...,https://www.amantis.net/limpin-limpiador-jugue...,False,False,True,True,False,True,True,False
1,22,NaN,NaN,NaN,NaN,NaN,Desliz! Lubricante íntimo de agua 100ml,NaN,tras muchos años de experiencia en lubricantes...,Bote de 100ml de venta exclusiva en amantis.ne...,https://www.amantis.net/desliz-lubricante-inti...,True,False,False,True,False,False,True,False
2,112,NaN,NaN,NaN,NaN,NaN,TOBOGANE HOT RABBIT,el superventas de amantis ¡mejorado!,vuelve nuestro vibrador de doble estimulacion ...,"Medidas: 19 cm (11 cm insertables) y 3,3 cm/ 2...",https://www.amantis.net/tobogane-hot-rabbit-el...,False,False,False,True,False,True,True,False
3,207,NaN,NaN,NaN,NaN,NaN,BALLENATO,tu vibrador a distancia con aleta móvil y sume...,de las profundidades mas humedas llega ballena...,Mando a distancia.14cm de punta a cola. 10cm l...,https://www.amantis.net/ballenato-tu-vibrador-...,False,False,False,True,False,True,True,False
4,412,NaN,NaN,NaN,NaN,NaN,Bacanal Gel Anal monodosis,pack de 10 uds,bacanal es un lubricante en gel de base de agu...,Contiene 3 ml x 10 unidadesLubricante íntimo a...,https://www.amantis.net/bacanal-monodosis-pack...,True,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,215,NaN,NaN,NaN,NaN,NaN,MYKONOS,conjunto lencero dulcemente sexy,"muchas veces cuanto mas sexies nos sentimos, m...","Talla: S, M, LMaterial: Nylon y spandexColor: ...",https://www.amantis.net/mykonos-conjunto-lence...,False,False,False,False,False,False,True,False
537,572,NaN,NaN,NaN,NaN,NaN,SUSU,conjunto lencero con volantes,el sexo es una de las cosas mas divertidas que...,"Talla: S, M, LMaterial: Nylon y spandexColor:...",https://www.amantis.net/susu-conjunto-lencero-...,False,False,False,False,False,False,True,False
538,449,NaN,NaN,NaN,NaN,NaN,COSQUI,cosquilleador erógeno,la finalidad del encuentro sexual es el placer...,NaN,https://www.amantis.net/cosqui-cosquilleador-e...,False,False,False,False,False,False,True,False
539,574,NaN,NaN,NaN,NaN,NaN,DITALE Vibrador para dedo,NaN,¿como podemos intensificar sensaciones? las co...,Vibrador de bolsillo con sujeciónMateriales: S...,https://www.amantis.net/ditale-vibrador-dedo/,False,False,False,False,False,True,True,False


In [87]:
df_tag_out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 541 entries, 0 to 540
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                541 non-null    int64 
 1   PRODUCT_          0 non-null      object
 2   SLOGAN_           0 non-null      object
 3   DESCRIPTION_      0 non-null      object
 4   CHARACTERISTICS_  0 non-null      object
 5   LISTA_URL_        0 non-null      object
 6   PRODUCT           541 non-null    object
 7   SLOGAN            464 non-null    object
 8   DESCRIPTION       541 non-null    object
 9   CHARACTERISTICS   492 non-null    object
 10  LISTA_URL         541 non-null    object
 11  amenities         541 non-null    bool  
 12  anal              541 non-null    bool  
 13  BDSM              541 non-null    bool  
 14  femenino          541 non-null    bool  
 15  masculino         541 non-null    bool  
 16  juguetes          541 non-null    bool  
 17  lenceria        

Procedemos igual con los ID de los USERS.

In [59]:
def mapeo_usuarios(BBDD, df_user):
   '''Función para la conversión de los ID de productos

   Input:
      - BBDD (str): nombre de la BBDD.

      - df_user (dataframe): Dataframe donde se encuentran los productos

   Output:
      - map_user (Dict): Diccionario con los ID mapeados coincidentes.
       
      - map_user_out (Dict): Diccionario con los ID mapeados no coincidentes.
   '''
   print("Vamos a reindexar los usuarios")


   '''Llamando a la base de datos para extraer información'''
   conn = sqlite3.connect(BBDD)
   cursor = conn.cursor()
   query='''SELECT ID,USERS FROM USERS'''
   df_bbdd=f.sql_query(query,cursor)
   conn.commit()
   cursor.close()
   conn.close()

   '''Obteniendo los ID coincidentes'''

   df_users_in = pd.merge(df_bbdd, df_user, left_on="USERS", right_on="USERS")
   id_user_BBDD = df_users_in['ID'].tolist()
   id_user_new = df_users_in['ID_USERS'].tolist()
   map_user = {k: v for k, v in zip(id_user_BBDD, id_user_new)}

   '''Obteniendo los ID NO coincidentes'''
   max_id_user=df_bbdd['ID'].max()
   df_users_out = pd.merge(df_bbdd, df_user, left_on="USERS", right_on="USERS",how='right',suffixes=('_',''))
   df_users_out = df_users_out[df_users_out['ID'].isnull()]
   id_user_out = df_users_out['ID_USERS'].tolist()
   df_users_out = df_users_out[df_user.columns]
   df_users_out['ID'] = max_id_user + 1 + df_users_out.index
   id_new_user = df_users_out['ID'].tolist()
   map_user_out = {k: v for k, v in zip(id_new_user,id_user_out )}
   map_user.update(map_user_out)

   return map_user, map_user_out

A partir de aqui procedemos al mapeado de los IDs correspondientes a cada dataframe y a la reducción del mismo en el caso de comments.

En cualquier caso, como hemos indicado antes en las tablas de *PRODUCTS*, *USERS*, *TAGS* solo se hará la ingesta de los datos nuevos, a partir de un **JOIN RIGHT**.

En el caso de *PRICES* se hará una ingesta total y en *COMMENTS* la ingesta indicada anteriormente (por fecha máxima de comentario en la BBDD).

In [31]:
BBDD="Resources/online_shop.db"
conn = sqlite3.connect(BBDD)
cursor = conn.cursor()
query_1='''SELECT ID,USERS FROM USERS'''
df_bbdd_user=f.sql_query(query_1,cursor)
query_2='''SELECT ID,URL FROM PRODUCT'''
df_bbdd_product=f.sql_query(query_2,cursor)
query_3='''SELECT * FROM TAGS'''
df_bbdd_tags=f.sql_query(query_3,cursor)

# conn.commit()
# cursor.close()
# conn.close()

In [32]:
df_bbdd_tags

,ID,JUGUETES,MUEBLES,LENCERIA,MASCULINO,FEMININO,ANAL,BDSM,AMENITIES
0,0,1,0,0,1,0,0,0,0
1,1,1,1,0,0,0,1,0,0
2,2,0,0,1,1,0,1,0,0
3,3,1,1,0,0,0,0,0,0
4,4,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
546,571,0,0,0,0,0,0,1,0
547,572,1,1,0,0,0,1,0,0
548,573,0,0,0,0,0,0,1,0
549,574,0,0,0,1,0,1,0,0


In [76]:
df_product=pd.read_csv(r'Data\productos_240216.csv')
df_tags=pd.read_csv(r'Data\tags_240216.csv')
df_users=pd.read_csv(r'Data\usuarios_240216.csv')
df_prices=pd.read_csv(r'Data\precios_240216.csv')
df_comment=pd.read_csv(r'Data\comentarios_240216.csv')


In [ ]:
'''Reducimos los datos de products y reindexamos'''
df_product_new = pd.merge(df_bbdd_product, df_product,left_on="URL", right_on="LISTA_URL",how='right',suffixes=('_',''))
df_product_new = df_product_new[df_product_new['URL'].isnull()]
df_product_new.replace({'ID':map_product},inplace=True)

'''Reducimos los datos de users y reindexamos'''
df_users_new = pd.merge(df_bbdd_user, df_users, left_on="USERS", right_on="USERS",how='right',suffixes=('_',''))
df_users_new = df_users_new[df_users_new['URL'].isnull()]
df_users_new.replace({'ID':map_user},inplace=True)

'''Reindexamos los datos de prices'''
df_prices_new=df_prices.replace({'ID':map_product},inplace=True)

'''Reducimos los datos de comments y reindexamos'''
df_comment_new=reengineer_comment("Resources/online_shop.db",df_comment)
df_comment_new.replace({'ID_PRODUCT':map_product},inplace=True)
df_comment_new.replace({'ID_USERS':map_user},inplace=True)

# '''Reducimos los datos de tags y reindexamos'''
df_tag.replace({'ID':map_product},inplace=True)
df_tag_out = pd.merge(df_product_out, df_tag,left_on="ID", right_on="ID",how='left',suffixes=('_',''))
df_tag_out=df_tag_out[nombre_listas]




Cargando los datos en la BBDD.

In [ ]:

lista_valores_product = df_product_new.values.tolist()
cursor.executemany("INSERT INTO PRODUCTS VALUES (?,?,?,?,?,?)", lista_valores_product)

lista_valores_users = df_users_new.values.tolist()
cursor.executemany("INSERT INTO USERS VALUES (?,?)", lista_valores_users)

lista_valores_prices = df_prices_new.values.tolist()
cursor.executemany("INSERT INTO PRICES VALUES (?,?,?,?,?)", lista_valores_prices)

lista_valores_comments = df_comment_new.values.tolist()
cursor.executemany("INSERT INTO COMMENTS VALUES (?,?,?,?,?,?)", lista_valores_comments)

# lista_valores_tags = df_tags_new.values.tolist()
# cursor.executemany("INSERT INTO TAGS VALUES (?,?,?,?,?,?,?,?,?)", lista_valores_tags)



conn.commit()
cursor.close()
conn.close()

Vamos a ver como generamos una función lo más generalista posible para hacer el mapeado.

Primeramente vamos a generar la función para realizar indexados de los dataframes *Product*, *TAGS* y *USERS*.

In [ ]:
def ingest_product_users (df_BBDD,df, join_left,join_right,how,map):
    '''
    Función para reducir el numero de registros del dataframe product o users.

    Input:
    - df_BBDD (pd.DataFrame): DataFrame de base de datos.
    - df (pd.DataFrame): DataFrame con registros a reducir.
    - join_left (str): Nombre de la columna de dataframe_BBDD donde hacer join.
    - join_right (str): Nombre de la columna de df donde hacer join.
    - how (str): Tipo de join.
    - map (Dict): Diccionario donde están la relación de indices a sustituir 

    Return:
    - df_new (pd.DataFrame): DataFrame con registros reindexados y reducidos.
    '''

    df_new = pd.merge(df_BBDD, df,left_on=join_left, right_on=join_right,how=how,suffixes=('_',''))
    df_new=df_new[df_new[join_left].isnull()]
    df_new.replace({'ID':map},inplace=True)
    return df_new

def ingest_tags (df_BBDD,df, join_left,join_right,how,ID,map,list):
    '''
    Función para reducir el numero de registros del dataframe product o users.

    Input:
    - df_BBDD (pd.DataFrame): DataFrame de base de datos.
    - df (pd.DataFrame): DataFrame con registros a reducir.
    - join_left (str): Nombre de la columna de dataframe_BBDD donde hacer join.
    - join_right (str): Nombre de la columna de df donde hacer join.
    - how (str): Tipo de join.
    - ID (str): indice a sustituir.
    - map (Dict): Diccionario donde están la relación de indices a sustituir 
    - list (list): Lista con los nombres de las columnas a permanecer.

    Return:
    - df_new (pd.DataFrame): DataFrame con registros reindexados y reducidos.
    '''

    df_new = pd.merge(df_BBDD, df,left_on=join_left, right_on=join_right,how=how,suffixes=('_',''))
    df_new=df_new[df_new[join_left].isnull()]
    df_new.replace({ID:map},inplace=True)
    df_new=df_new[list]
    return df_new



In [ ]:
def reindex_2 (df,ID,map):
    df_new=df.replace({ID:map},inplace=True)
    return df_new


Ahora vamos a intentar generar una función para que independientemente del tamaño del dataframe/TABLA se pueda utilizar para ingestar los datos en la BBDD.

In [ ]:
def ingesta_datos (df,BBDD,TABLE):

    conn = sqlite3.connect(BBDD)
    cursor = conn.cursor()

    num= len(df.columns)
    columnas=["?" for _ in range(num)]
    columnas_total=", ".join(columnas)

    lista_valores=df.values.tolist()
    cursor.executemany(f"INSERT INTO {TABLE} VALUES (columntas_total)",lista_valores)

    print (f"Se han ingresado los datos a la tabla {TABLE}")

    conn.commit()
    cursor.close()
    conn.close()

In [61]:
map_product, map_product_out=mapeo_productos("Resources/online_shop.db", df_product)
map_user, map_user_out=mapeo_usuarios("Resources/online_shop.db", df_users)


Vamos a reindexar los productos
Vamos a reindexar los productos


In [90]:
'''Reducimos los datos de products y reindexamos'''
df_product_new = pd.merge(df_bbdd_product, df_product,left_on="URL", right_on="LISTA_URL",how='right',suffixes=('_',''))
df_product_new = df_product_new[df_product_new['URL'].isnull()]
df_product_new.replace({'ID':map_product},inplace=True)

'''Reducimos los datos de users y reindexamos'''
df_users_new = pd.merge(df_bbdd_user, df_users, left_on="USERS", right_on="USERS",how='right',suffixes=('_',''))
df_users_new = df_users_new[df_users_new['ID'].isnull()]
df_users_new.replace({'ID':map_user},inplace=True)

'''Reindexamos los datos de prices'''
df_prices_new=df_prices.replace({'ID':map_product},inplace=True)

'''Reducimos los datos de comments y reindexamos'''
df_comment_new=reengineer_comment("Resources/online_shop.db",df_comment)
df_comment_new.replace({'ID_PRODUCT':map_product},inplace=True)
df_comment_new.replace({'ID_USERS':map_user},inplace=True)

'''Reducimos los datos de tags y reindexamos'''
df_tags.replace({'ID':map_product},inplace=True)
df_tag_out = pd.merge(df_product_out, df_tags,left_on="ID", right_on="ID",how='right',suffixes=('_',''))
df_tag_out=df_tag_out[nombre_listas]

Iniciando la conversión de fecha y la reducción de datos de comentarios
El número de registros a ingresar es: 873


C:\Users\Javier\AppData\Local\Temp\ipykernel_18424\389071867.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comment_new.replace({'ID_USERS':map_user},inplace=True)


In [91]:
df_tag_out

,amenities,anal,BDSM,femenino,masculino,juguetes,lenceria,muebles,ID
0,False,False,True,True,False,True,True,False,41
1,True,False,False,True,False,False,True,False,97
2,False,False,False,True,False,True,True,False,112
3,False,False,False,True,False,True,True,False,207
4,True,True,False,False,False,False,True,False,412
...,...,...,...,...,...,...,...,...,...
536,False,False,False,False,False,False,True,False,133
537,False,False,False,False,False,False,True,False,572
538,False,False,False,False,False,False,True,False,441
539,False,False,False,False,False,True,True,False,574
